# Indigenous lands on which observatories are located

In [19]:
import pandas as pd
import folium
import numpy as np
import math
from astropy.coordinates import EarthLocation as EL
import astropy.units as u
import requests

In [20]:
sites = EL.get_site_names()

## Get map

In [21]:
folium_map = folium.Map(location=[43.6532, -79.3832],
                        zoom_start=3,
                        tiles="https://tiles.stadiamaps.com/tiles/outdoors/{z}/{x}/{y}{r}.png",attr='http://openstreetmap.org')

In [22]:
EL.of_site(sites[3]).geodetic

GeodeticLocation(lon=<Longitude -67.755 deg>, lat=<Latitude -23.029 deg>, height=<Quantity 5000. m>)

In [23]:
sites

['',
 '',
 '',
 'ALMA',
 'ATST',
 'Anglo-Australian Observatory',
 'Apache Point',
 'Apache Point Observatory',
 'Atacama Large Millimeter Array',
 'BAO',
 'BBSO',
 'Beijing XingLong Observatory',
 'Black Moshannon Observatory',
 'CHARA',
 'Canada-France-Hawaii Telescope',
 'Catalina Observatory',
 'Cerro Pachon',
 'Cerro Paranal',
 'Cerro Tololo',
 'Cerro Tololo Interamerican Observatory',
 'DCT',
 'DKIST',
 'Discovery Channel Telescope',
 'Dominion Astrophysical Observatory',
 'G1',
 'GBT',
 'GEO',
 'GEO_600',
 'Gemini South',
 'Green Bank Telescope',
 'H1',
 'Hale Telescope',
 'Haleakala Observatories',
 'Happy Jack',
 'IAO',
 'JCMT',
 'James Clerk Maxwell Telescope',
 'Jansky Very Large Array',
 'K1',
 'KAGRA',
 'Keck Observatory',
 'Kitt Peak',
 'Kitt Peak National Observatory',
 'L1',
 'LHO',
 'LHO_4k',
 'LLO',
 'LLO_4k',
 'La Silla Observatory',
 'Large Binocular Telescope',
 'Las Campanas Observatory',
 'Lick Observatory',
 'Lowell Observatory',
 'MWA',
 'Manastash Ridge Observ

In [24]:
for site in sites:
    loc = EL.of_site(site).geodetic
    locstr = '{:.6f},{:.6f}'.format(loc[1].to(u.deg).value, loc[0].to(u.deg).value)
#     print("https://native-land.ca/api/index.php?map=languages&position={:s}".format(locstr))

    response = requests.get("https://api.native-land.ca/index.php?maps=territories&position={:s}".format(locstr))
    infos = response.json()
    namelist = []
    for info in infos:
        namelist.append(info['properties']['Name'])

    namestr = ''
    c = 0
    for name in namelist:
        if c==(len(namelist)-1):
            namestr=namestr+"{:s}".format(name)
        else:
            namestr=namestr+"{:s},\n".format(name)
        c+=1
    popup_text = "Observatory Name: {}<br> Indigenous Land: {}"
    popup_text = popup_text.format(site,
                          namestr)

    folium.CircleMarker(location = [loc[1].to(u.deg).value, loc[0].to(u.deg).value],fill=True, popup=popup_text).add_to(folium_map)


#     ,radius=radius, color=color,popup=popup_text

In [25]:
folium_map